In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,32,5)
        self.conv2=nn.Conv2d(32,64,5)
        self.conv3=nn.Conv2d(64,128,5)
        self.fc1=nn.Linear(9*9*128,512)
        self.fc2=nn.Linear(512,2)

    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=F.max_pool2d(x,2,2)
        x=F.relu(self.conv2(x))
        x=F.max_pool2d(x,2,2)
        x=F.relu(self.conv3(x))
        x=F.max_pool2d(x,2,2)
        x=x.view(-1,9*9*128)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.softmax(x,dim=1)
PATH = "model_state_dict.pt"
model= Net()
model.load_state_dict(torch.load(PATH))
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=10368, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [2]:
import numpy as np

def predict_from_image(img,net):
    try:
        image=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        image=cv2.resize(image,(100,100))
        image=np.array(image)
        image=torch.from_numpy(image)
        image=image.view(-1,1,100,100)
        image=image/255.0
        res=net(image)
        pred=torch.argmax(res)
        if(pred.item()==1):
            return True
        else:
            return False
    except:
        return


In [11]:
import math
videoFile = "test_videos/29.wmv"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        res=predict_from_image(frame,model)
        if(res==True):
            cv2.imwrite("new_tests/L-shape/frame{}29.jpg".format(str(int(frameId))), frame)
        elif(res==False):
            cv2.imwrite("new_tests/no-L-shape/frame{}29.jpg".format(str(int(frameId))), frame)
cap.release()
print ("Done!")



Done!
